#**Trabajo Práctico N° 1 - Parte 1**
# **Procesos**

##Enunciado:
###*Generar* el siguiente árbol de procesos en los lenguajes: C, Java y Python.


###**Lenguaje Python**

In [ ]:
%%writefile procesos.py
import os
import sys
import time


def main():
    print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: A")
    time.sleep(5)

    pid = os.fork()

    if pid < 0:
        sys.exit("Error al crear el primer proceso.")

    if pid == 0: # A crea B
        print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: B")
        time.sleep(5)

        pid = os.fork()
        if pid == 0: # B crea C
            print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: C")
            time.sleep(5)

            pid = os.fork()
            if pid == 0: # C crea E
                print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: E")
                time.sleep(5)

                pid = os.fork()
                if pid == 0: # E crea H
                    print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: H")
                    time.sleep(5)
                else:
                    pid = os.fork()
                    if pid == 0: # E crea I
                        print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: I")
                        time.sleep(5)
        else: # B crea D
            pid = os.fork()
            if pid == 0:
                print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: D")
                time.sleep(5)

                pid = os.fork()
                if pid == 0: # D crea F
                    print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: F")
                    time.sleep(5)
                else:
                    pid = os.fork()
                    if pid == 0: # D crea G
                        print(f"PID: {os.getpid()} PPID: {os.getppid()} Letra: G")
                        time.sleep(5)

    while True:
        try:
            os.wait()
        except ChildProcessError:
            break


if __name__ == "__main__":
    main()

Writing procesos.py


In [ ]:
!chmod 700 ./procesos.py

Ejecuto en segundo plano el script de Python, para que mientras se este ejecutando este script se pueda usar el comando ps y pstree

In [ ]:
!nohup python procesos.py 1>salidaPython 2>/dev/null &

Compruebo su ejecución y árbol de procesos con pstree

In [ ]:
!ps -ef | grep procesos.py


root        7338     458  0 16:38 ?        00:00:00 /bin/bash -c ps -ef | grep procesos.py
root        7340    7338  0 16:38 ?        00:00:00 grep procesos.py


In [ ]:
!pstree -pc

python3(7019)───python3(7047)───python3(7075)───python3(7103)─┬─python3(7122)
                                                              └─python3(7123)


###**Lenguaje C**

In [ ]:
%%writefile procesos.c
#include <stdio.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <stdlib.h>

#define TIME_WAIT 5

int main(){
    pid_t pid_B, pid_C, pid_D, pid_E, pid_F, pid_G, pid_H, pid_I;

    printf("Process A (PID: %d)\n", getpid());
    sleep(TIME_WAIT);
    pid_B = fork(); // A crea B
    if (pid_B == 0) {
        printf("Process B (PID: %d, Parent: %d)\n", getpid(), getppid());
        sleep(TIME_WAIT);

        pid_C = fork(); // B crea C

        if (pid_C == 0) {
            printf("Process C (PID: %d, Parent: %d)\n", getpid(), getppid());
            sleep(TIME_WAIT);

            pid_E = fork(); // C crea E
            if (pid_E == 0) {
                printf("Process E (PID: %d, Parent: %d)\n", getpid(), getppid());
                sleep(TIME_WAIT);

                pid_H = fork(); // E crea H
                if (pid_H == 0) {
                    printf("Process H (PID: %d, Parent: %d)\n", getpid(), getppid());
                    sleep(TIME_WAIT);
                    return EXIT_SUCCESS; // Detener H
                }

                pid_I = fork(); // E crea I
                if (pid_I == 0) {
                    printf("Process I (PID: %d, Parent: %d)\n", getpid(), getppid());
                    sleep(TIME_WAIT);
                    return EXIT_SUCCESS; // Detener I
                }

                wait(NULL); // Esperar a H e I
                wait(NULL); // Esperar a H e I
                return EXIT_SUCCESS; // Detener E
            }

            wait(NULL); // Esperar a E
            return EXIT_SUCCESS; // Detener C
        }

        pid_D = fork(); // B crea D
        if (pid_D == 0) {
            printf("Process D (PID: %d, Parent: %d)\n", getpid(), getppid());
            sleep(TIME_WAIT);

            pid_F = fork(); // D crea F
            if (pid_F == 0) {
                printf("Process F (PID: %d, Parent: %d)\n", getpid(), getppid());
                sleep(TIME_WAIT);
                return EXIT_SUCCESS; // Detener F
            }

            pid_G = fork(); // D crea G
            if (pid_G == 0) {
                printf("Process G (PID: %d, Parent: %d)\n", getpid(), getppid());
                sleep(TIME_WAIT);
                return EXIT_SUCCESS; // Detener G
            }

            wait(NULL); // Esperar a F y G
            wait(NULL); // Esperar a F y G
            return EXIT_SUCCESS; // Detener D
        }

        wait(NULL); // Esperar a C y D
        wait(NULL); // Esperar a C y D
        return EXIT_SUCCESS; // Detener B
    }

    wait(NULL); // Esperar a B
    return EXIT_SUCCESS; // Detener A
}

Writing procesos.c


Compilar el codigo C

In [ ]:
!gcc -o procesos.bin procesos.c

Este programa se ejecuta en segundo plano, permitiendo que, mientras se está ejecutando, se puedan utilizar los comandos 'ps' y 'pstree' para visualizar su proceso en el sistema.

In [ ]:
!nohup ./procesos.bin 1>salidaC 2>/dev/null &

In [ ]:
!ps -ef | grep procesos.bin

root       13189       1  0 15:58 ?        00:00:00 ./procesos.bin
root       13198    3548  0 15:58 ?        00:00:00 /bin/bash -c ps -ef | grep procesos.bin
root       13200   13198  0 15:58 ?        00:00:00 grep procesos.bin


In [ ]:
!pstree -pc

procesos.bin(13189)───procesos.bin(13213)───procesos.bin(13236)───procesos.bin(13262)─┬─procesos.bi+
                                                                                      └─procesos.bi+


In [ ]:
!pstree -p | grep procesos.bin

###**Lenguaje Java**

Instalar y configurar Java en Colab (si aún no está instalado)

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!java -version

openjdk version "11.0.23" 2024-04-16
OpenJDK Runtime Environment (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [ ]:
%%writefile childProcess.java
import java.io.IOException;

public class ChildProcess {

    public static void main(String[] args) throws IOException, InterruptedException {
        Process[] childs;
        char[] letters;

        switch (args[0]) {
            case "B":
                System.out.println("PID: " + getPID() + " PPID: " + getPPID() + " Letra: " + args[0]);
                Thread.sleep(10000);
                letters = new char[] { 'C', 'D' };
                childs = new Process[letters.length];
                for (int i = 0; i < letters.length; i++) {
                    childs[i] = createChild(letters[i]);
                }
                waitChilds(letters.length, childs);
                System.exit(0);
            case "C":
                System.out.println("PID: " + getPID() + " PPID: " + getPPID() + " Letra: " + args[0]);
                Thread.sleep(10000);
                letters = new char[] { 'E' };
                childs = new Process[letters.length];
                for (int i = 0; i < letters.length; i++) {
                    childs[i] = createChild(letters[i]);
                }
                waitChilds(letters.length, childs);
                System.exit(0);
            case "D":
                System.out.println("PID: " + getPID() + " PPID: " + getPPID() + " Letra: " + args[0]);
                Thread.sleep(10000);
                letters = new char[] { 'F', 'G' };
                childs = new Process[letters.length];
                for (int i = 0; i < letters.length; i++) {
                    childs[i] = createChild(letters[i]);
                }
                waitChilds(letters.length, childs);
                System.exit(0);
            case "E":
                System.out.println("PID: " + getPID() + " PPID: " + getPPID() + " Letra: " + args[0]);
                Thread.sleep(10000);
                letters = new char[] { 'H', 'I' };
                childs = new Process[letters.length];
                for (int i = 0; i < letters.length; i++) {
                    childs[i] = createChild(letters[i]);
                }
                waitChilds(letters.length, childs);
                System.exit(0);
            case "F":
            case "G":
            case "H":
            case "I":
                System.out.println("PID: " + getPID() + " PPID: " + getPPID() + " Letra: " + args[0]);
                Thread.sleep(10000);
                System.exit(0);
            default:
                break;
        }
    }

    public static long getPID() {
        return ProcessHandle.current().pid();
    }

    public static long getPPID() {
        return ProcessHandle.current().parent().get().pid();
    }

    public static Process createChild(char letter) throws IOException {
        ProcessBuilder builder = new ProcessBuilder("java", "/content/ChildProcess.java",
                Character.toString(letter));
        ;
        builder.inheritIO();
        return builder.start();
    }

    public static void waitChilds(int numberOfChilds, Process[] childs) throws InterruptedException {
        for (int i = 0; i < numberOfChilds; i++) {
            childs[i].waitFor();
        }
    }
}

Overwriting childProcess.java


In [ ]:
%%writefile parentProcess.java
import java.io.IOException;

public class ParentProcess {

    public static void main(String[] args) throws IOException, InterruptedException {
        Process[] childs;
        char[] letters;

        System.out.println("PID: " + getPID() + " PPID: " + getPPID() + " Letra: A");
        Thread.sleep(10000);
        letters = new char[] { 'B' };
        childs = new Process[letters.length];
        for (int i = 0; i < letters.length; i++) {
            childs[i] = createChild(letters[i]);
        }
        waitChilds(letters.length, childs);
        System.exit(0);
    }

    public static long getPID() {
        return ProcessHandle.current().pid();
    }

    public static long getPPID() {
        return ProcessHandle.current().parent().get().pid();
    }

    public static Process createChild(char letter) throws IOException {
        ProcessBuilder builder = new ProcessBuilder("java", "/content/ChildProcess.java",
                Character.toString(letter));
        builder.inheritIO();
        return builder.start();
    }

    public static void waitChilds(int numberOfChilds, Process[] childs) throws InterruptedException {
        for (int i = 0; i < numberOfChilds; i++) {
            childs[i].waitFor();
        }
    }
}

Overwriting parentProcess.java


In [ ]:
!nohup java parentProcess.java 1>salidaJava 2>/dev/null &

In [ ]:
!ps -ef | grep parentProcess.java

root        7588       1 99 16:39 ?        00:00:02 java parentProcess.java
root        7614     458  0 16:39 ?        00:00:00 /bin/bash -c ps -ef | grep parentProcess.java
root        7616    7614  0 16:39 ?        00:00:00 grep parentProcess.java


In [ ]:
!ps -ef | grep childProcess.java

root        7573     458  0 16:39 ?        00:00:00 /bin/bash -c ps -ef | grep childProcess.java
root        7575    7573  0 16:39 ?        00:00:00 grep childProcess.java


In [ ]:
!pstree -p | grep java

docker-init(1)-+-java(7588)-+-{java}(7589)
               |            |-{java}(7590)
               |            |-{java}(7591)
               |            |-{java}(7592)
               |            |-{java}(7593)
               |            |-{java}(7594)
               |            |-{java}(7595)
               |            |-{java}(7596)
               |            |-{java}(7597)
               |            |-{java}(7600)
               |            |-{java}(7601)
               |            |-{java}(7603)
               |            |-{java}(7604)
               |            |-{java}(7606)
               |            |-{java}(7607)
               |            |-{java}(7608)
               |            `-{java}(7609)
